<a href="https://colab.research.google.com/github/JuanParias29/Titanic_Survival_Model/blob/main/Titanic_LogisticModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller Modelos Lineales

In [ ]:
# Instalación de la librería ydata-profiling para EDA
!pip install ydata-profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 673.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.1/400.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.0/679.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.2 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=13e9effb8c1d97ddecc4432be22fdbfe06952ac1a4123bff1ed49b9318dcacee
  Stored in directory: /root/.cache/pip/wheels/8d/55/1a/19cd535375ed1ede0c996405ebffe34b196d78e2d9545723a2
Successfully built htmlmin
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.0
    Uninstalling scipy-1.16.0:
      

In [ ]:
!pip install gradio

In [ ]:
import os
import pandas as pd
import numpy as np
import gradio as gr

from ydata_profiling import ProfileReport
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/refs/heads/master/titanic.csv'
titanic = pd.read_csv(url)

In [ ]:
report = ProfileReport(titanic)
report.to_file(output_file="Reporte.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 12/12 [00:00<00:00, 39.50it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### **Solucion Preguntas Ejercicio 1**

1-**¿Cuantas filas tiene el archivo?**

La base de datos cuenta con 891 registros

2-**¿Cuantas variables de tipo categórico tiene?**

La base de datos cuenta con 4 variables de tipo categorico (Survived ,Pclass ,Sex ,Embarked).

3-**¿Cuáles son las 3 palabras más comunes en la columna Name?**

Las 3 palabras mas comunes son Miss,Mrs y Mr que corresponden a títulos de cortesía en inglés que se colocan antes del nombre de una persona.

4-**¿Cuáles son las variables más correlacionadas con variable Survived? ¿Tiene sentido esta correlación?**

Las variables que estan mas correlacionadas a Survived son; Sex (Sexo), Pclass (Clase asignada al pasajero) y Fare (Tarifa). Tienen sentido ya que puede deberse a politicas de evacuacion del barco o preferencias sociales.

In [ ]:
titanic.isna().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


###**Limpieza de datos**

A partir de lo observado en el EDA podemos identificar algunas de las variables que contienen datos nulos entre ellas esta "Age", "Cabin", "Embarked".

Debido a que "Cabin" y "Embarked" son variables categoricas seran eliminadas junto con las demas categoricas. Para el caso particular de
"Age" se tiene que el 20% de los datos son nulos, es entonces que se procede a evaluar.

In [ ]:
titanic_clear = titanic
titanic_clear.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
columns = ['PassengerId','Pclass', 'Name', 'Ticket','Age','Cabin','Embarked']
titanic_clear = titanic.drop(columns, axis=1)
#Male -> 1 - Female -> 0
titanic_clear['Sex'].replace(["male", "female"] , [1, 0], inplace=True)

/tmp/ipython-input-10-4158289046.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  titanic_clear['Sex'].replace(["male", "female"] , [1, 0], inplace=True)
/tmp/ipython-input-10-4158289046.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  titanic_clear['Sex'].replace(["male", "female"] , [1, 0], 

In [ ]:
titanic_clear.head()

,Survived,Sex,SibSp,Parch,Fare
0,0,1,1,0,7.2500
1,1,0,1,0,71.2833
2,1,0,0,0,7.9250
3,1,0,1,0,53.1000
4,0,1,0,0,8.0500


In [ ]:
titanic_clear.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Sex       891 non-null    int64  
 2   SibSp     891 non-null    int64  
 3   Parch     891 non-null    int64  
 4   Fare      891 non-null    float64
dtypes: float64(1), int64(4)
memory usage: 34.9 KB


In [ ]:
y = titanic_clear['Survived']
X = titanic_clear[['Sex',  'SibSp',        'Parch',        'Fare']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

LogisticRegression()

In [ ]:
y_pred = modelo.predict(X_test)
print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))

Matriz de Confusión:
[[134  23]
 [ 33  78]]


In [ ]:
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred, digits=3))


Reporte de Clasificación:
              precision    recall  f1-score   support

           0      0.802     0.854     0.827       157
           1      0.772     0.703     0.736       111

    accuracy                          0.791       268
   macro avg      0.787     0.778     0.782       268
weighted avg      0.790     0.791     0.789       268



In [ ]:
# Función de predicción para Gradio
def predecir_supervivencia(sex, sibsp, parch, fare):
    entrada = pd.DataFrame([[sex, sibsp, parch, fare]], columns=['Sex', 'SibSp', 'Parch', 'Fare'])
    prob = modelo.predict_proba(entrada)[0][1]  # Probabilidad de sobrevivir (clase 1)
    return f"Probabilidad de sobrevivencia: {prob:.2%}"

# Definir interfaz
iface = gr.Interface(
    fn=predecir_supervivencia,
    inputs=[
        gr.Radio(choices=[0, 1], label="Sexo (0 = Mujer, 1 = Hombre)"),
        gr.Number(label="Número de hermanos/esposos a bordo (SibSp)"),
        gr.Number(label="Número de padres/hijos a bordo (Parch)"),
        gr.Number(label="Tarifa pagada (Fare)")
    ],
    outputs="text",
    title="Predicción de Supervivencia Titanic",
    description="Introduce los datos del pasajero para estimar su probabilidad de supervivencia."
)

iface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://38371a280640a471f5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
